In [ ]:
import scipy.signal
import numpy as np
import cv2

In [ ]:
def mat2gray(mat):
    maxv  = mat.max()
    minv  = mat.min()
    gap = maxv - minv
    return (mat-minv)/gap

In [ ]:
def DaugmanIrisCore(ringTemplates, imggrey):

    candidateImage = imggrey
    covstack = np.zeros(list(candidateImage.shape) + [len(ringTemplates)])

#    results = [DaugmanIrisCore.pool.apply_async(scipy.signal.fftconvolve,(candidateImage,template,'same')) for template in ringTemplates]
#    parallelResults = [r.get() for r in results]
#    for i in range(len(parallelResults)):
#        covstack[:,:,i] = parallelResults[i][0]
    
    for i in range(len(ringTemplates)):
        #通过fft来做卷积,时间复杂度从2次方变成nlogn
        covstack[:,:,i] = scipy.signal.fftconvolve(candidateImage,ringTemplates[i],'same')
    #沿着第二个维度取一阶差值？
    covstackdiff = np.diff(covstack,1,2);
    covstackdiff = np.concatenate((np.zeros(list(imggrey.shape) +[1]),covstackdiff),2)
    covstackdiff = scipy.signal.lfilter(np.ones(5)/5,1,covstackdiff)
    covstackdiffmax, covstackdiffmaxRadius = covstackdiff.max(-1),np.argmax(covstackdiff,-1)
    covstackdiffmax = mat2gray(covstackdiffmax)
    covstackdiffmaxRadius = np.float64(covstackdiffmaxRadius)
    covstackdiffmaxRadius[(covstackdiffmaxRadius==0).nonzero()] = np.nan
    return covstackdiffmax,covstackdiffmaxRadius,covstackdiff

In [ ]:
def DaugmanIrisCore2(ringTemplates, imggrey,countStr,numStr):

    candidateImage = imggrey
    covstack = np.zeros(list(candidateImage.shape) + [len(ringTemplates)])

#    results = [DaugmanIrisCore.pool.apply_async(scipy.signal.fftconvolve,(candidateImage,template,'same')) for template in ringTemplates]
#    parallelResults = [r.get() for r in results]
#    for i in range(len(parallelResults)):
#        covstack[:,:,i] = parallelResults[i][0]
    
    for i in range(len(ringTemplates)):
        #通过fft来做卷积,时间复杂度从2次方变成nlogn
        covstack[:,:,i] = scipy.signal.fftconvolve(candidateImage,ringTemplates[i],'same')
    #沿着第二个维度取一阶差值？
    covstackdiff = np.diff(covstack,1,2);
    covstackdiff = np.concatenate((np.zeros(list(imggrey.shape) +[1]),covstackdiff),2)
    covstackdiff = scipy.signal.lfilter(np.ones(5)/5,1,covstackdiff)
    covstackdiffmax, covstackdiffmaxRadius = covstackdiff.max(-1),np.argmax(covstackdiff,-1)
    covstackdiffmax = mat2gray(covstackdiffmax)
    #cv2.imshow('covstackdiffmax',covstackdiffmax)
    print 'covstackdiffmax'
    print type(covstackdiffmax)
    print sum(sum(covstackdiffmax))
    cv2.imwrite('/Users/ly/MyGithub/eyeProject/learnProcess/learnPython/max/max'+countStr+'_'+numStr+'.jpg',covstackdiffmax*255)
    covstackdiffmaxRadius = np.float64(covstackdiffmaxRadius)
    covstackdiffmaxRadius[(covstackdiffmaxRadius==0).nonzero()] = np.nan
    res=[]
    res.append(covstackdiffmax)
    res.append(covstackdiffmaxRadius)
    res.append(covstackdiff)
    return res